In [ ]:
dbutils.library.restartPython()

In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkFiles
from pyspark.sql import DataFrame
import pyspark.sql.functions as F

In [ ]:
app_name = "SharkTank"
spark = SparkSession.builder.appName(app_name).getOrCreate()

In [ ]:
pitch_data_file = "/FileStore/tables/S1_8Sharktankpitchesdeals.csv"
pitch_data_file

In [ ]:
df_pitches = spark.read \
  .format("com.databricks.spark.csv")\
  .options(header='true', inferSchema="true")\
  .load(pitch_data_file)
df_pitches.printSchema()

In [ ]:
import boto3

secret_name = "ut/finalproject/sharktank"
region_name = "us-east-2"
access_key = "ACCESS KEY"
secret_key = "SECRET ACCESS KEY"

session = boto3.session.Session(aws_access_key_id=access_key, aws_secret_access_key=secret_key, region_name=region_name)
client = session.client('secretsmanager')
secret_value = client.get_secret_value(SecretId=secret_name)
# secret_value

In [ ]:
import json
def get_connection(secret_value):
  return json.loads(secret_value['SecretString'])
# get_connection(secret_value)

In [ ]:
connection = get_connection(secret_value)

# Postgres credentials
jdbcHostname = connection['host']
jdbcPort = connection['port']
jdbcDatabase = "postgres"
dialect = "postgresql"
jdbcUsername = connection['username']
jdbcPassword = connection['password']

jdbcUrl = f"jdbc:{dialect}://{jdbcHostname}:{jdbcPort}/{jdbcDatabase}"
connectionProperties = {
  "user" : jdbcUsername,
  "password" : jdbcPassword,
  "driver" : "org.postgresql.Driver"
}
# for mysql driver = com.mysql.jdbc.Driver

In [ ]:
jdbcUrl

In [ ]:
connectionProperties

In [ ]:
table = "user_data"
mode = "overwrite"
df_pitches.write.jdbc(jdbcUrl, table, mode, connectionProperties)

In [ ]:
df = spark.read.jdbc(url=jdbcUrl, table=table, properties=connectionProperties)
display(df)